# UTSA CS 3793: Assignment-2

**Palacios - Jayden - (ubc007)**






## Learning Objectives



*   Game Playing
*   Chess - Board Setup & Rules
*   Adversarial Search
*   AI - Random vs MinMax



## Description

This assignment is focused on **game playing** and creating a proper **AI for chess**. 
In the following sections, you will complete a series of tasks to create a chess game board, rules for each chess piece, a Random AI and a MinMax AI that plays a game of chess for both players (white and black).

The base structure of the code is provided. You are supposed to write code for each of the functions. Comments are provided on what should be done. You **CANNOT** use a complete chess library and change the base code structure completely. However, you **CAN** change the code layout and name/format of the functions.


#Chess Board Setup & Rules

In this section, you will write code to import the necessary libraries and create:

1.   **ChessBoard** - This part will contain code to initialize the board, draw the board, get the board state and move piece.
2.   **ChessRules** - This part will contain code for the chess rules for each piece.



##Import Libraries

The code here will contain only **import** statements. A base list of the required libraries are already imported. You will most likely not need any other libraries, but if needed, add the import statements here. As mentioned before, you can not use any premade chess libraries.

In [1]:
# your code goes here
import string
import random
import os
import sys
import time
from IPython.display import clear_output


##ChessBoard

(10 points)

Fill the code in the code structure provided below for the ChessBoard. The main use of this code block write functions to initialize the board, draw the board, get the board state and move piece. You can add any other functions if needed.


In [2]:
#classes and other stuff i made to help
PGNsquares = [
        ['a8', 'b8', 'c8', 'd8', 'e8', 'f8', 'g8', 'h8'],
        ['a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7'],
        ['a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6'],
        ['a5', 'b5', 'c5', 'd5', 'e5', 'f5', 'g5', 'h5'],
        ['a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4', 'h4'],
        ['a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3'],
        ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2'],
        ['a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1', 'h1'],
]

def convertMovesToPGN(moves):
    round = 1
    pawnPromote = False
    pgnMoves = []
    for i in range(0, len(moves), 2):
        whiteMove = moves[i]
        roundLine = []
        roundLine.append("%d. " % (round))
        if whiteMove[0] == 'short':
            roundLine.append("O-O ")
        elif whiteMove[0] == 'long':
            roundLine.append("O-O-O ")
        else:
            if whiteMove[1] != 'P':
                if whiteMove[1] == 'T':
                    roundLine.append("N")
                else:
                    roundLine.append(whiteMove[1].upper())
            if whiteMove[2] != '.':
                if whiteMove[1] == 'P':
                    roundLine.append(PGNsquares[whiteMove[0][0][0]][whiteMove[0][0][1]][0])
                    if whiteMove[0][1][0] == 0:
                        pawnPromote = True
                        tmp = "=Q "
                roundLine.append("x")
            else:
                roundLine.append(PGNsquares[whiteMove[0][0][0]][whiteMove[0][0][1]][0])
            if pawnPromote:
                roundLine.append("%s" % (PGNsquares[whiteMove[0][1][0]][whiteMove[0][1][1]]))
                roundLine.append(tmp)
                pawnPromote = False
            else: 
                roundLine.append("%s " % (PGNsquares[whiteMove[0][1][0]][whiteMove[0][1][1]]))
        if i+1 == len(moves):
            pgnMoves.append(''.join(roundLine))
            break
        blackMove = moves[i+1]
        if whiteMove[0] == 'short':
            roundLine.append("O-O ")
        elif whiteMove[0] == 'long':
            roundLine.append("O-O-O ")
        else:
            if blackMove[1] != 'p':
                if blackMove[1] == 't':
                    roundLine.append("N")
                else:
                    roundLine.append(blackMove[1].upper())
            if blackMove[2] != '.':
                if blackMove[1] == 'p':
                    roundLine.append(PGNsquares[blackMove[0][0][0]][blackMove[0][0][1]][0])
                    if whiteMove[0][1][0] == 7:
                        pawnPromote = True
                        tmp = "=Q "
                roundLine.append("x")
            else:
                roundLine.append(PGNsquares[blackMove[0][0][0]][blackMove[0][0][1]][0])
            if pawnPromote:
                roundLine.append("%s" % (PGNsquares[blackMove[0][1][0]][blackMove[0][1][1]]))
                roundLine.append(tmp)
                pawnPromote = False
            else :
                roundLine.append("%s " % (PGNsquares[blackMove[0][1][0]][blackMove[0][1][1]]))
            pgnMoves.append(''.join(roundLine))
        round += 1
    return ''.join(pgnMoves)

#class to keep track of whether king can castle or not
#in the future this could be expanded as well as pieces having their own
#classes
class Chess:
    def __init__(self):
        self.board = []
        self.whiteKingMoved = False
        self.blackKingMoved = False
        self.wLeftRookMoved = False
        self.wRightRookMoved = False
        self.bLeftRookMoved = False
        self.bRightRookMoved = False
        
#piece tables for evl() found as a suggestion on stack over flow in order to give agent "positional awareness"
# links - https://www.chessprogramming.org/Simplified_Evaluation_Function
# https://stackoverflow.com/questions/17379849/simple-minimax-evaluation-function-for-chess-position

pawns = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [10, 10, 20, 40, 40, 20, 10, 10],
    [5, 5, 10, 30, 30, 10, 5, 5],
    [0, 0, 0, 25, 25, 0, 0, 0],
    [5, -5, -10, 0, 0, -10, -5, 5],
    [5, 10, 10, -20, -20, 10, 10, 5],
    [0,  0,  0,  0,  0,  0,  0,  0]
]

knights = [
    [-50,-40,-30,-30,-30,-30,-40,-50],
    [-40,-20, 0, 0, 0, 0,-20,-40],
    [-30, 0, 10, 15, 15, 10, 0,-30],
    [-30, 5, 15, 20, 20, 15, 5,-30],
    [-30, 0, 15, 20, 20, 15, 0,-30],
    [-30, 5, 10, 15, 15, 10, 5,-30],
    [-40,-20, 0, 5, 5, 0,-20,-40],
    [-50,-40,-30,-30,-30,-30,-40,-50]
]

bishops = [
    [-20,-10,-10,-10,-10,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5, 10, 10,  5,  0,-10],
    [-10,  5,  5, 10, 10,  5,  5,-10],
    [-10,  0, 10, 10, 10, 10,  0,-10],
    [-10, 10, 10, 10, 10, 10, 10,-10],
    [-10,  5,  0,  0,  0,  0,  5,-10],
    [-20,-10,-10,-10,-10,-10,-10,-20]
]

rooks = [
    [0,  0,  0,  0,  0,  0,  0,  0],
    [5, 10, 10, 10, 10, 10, 10,  5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [0,  0,  0,  5,  5,  5,  0,  0]
]

queens = [
    [-20,-10,-10, -5, -5,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5,  5,  5,  5,  0,-10],
    [ -5,  0,  5,  5,  5,  5,  0, -5],
    [  0,  0,  5,  5,  5,  5,  0, -5],
    [-10,  5,  5,  5,  5,  5,  0,-10],
    [-10,  0,  5,  0,  0,  0,  0,-10],
    [-20,-10,-10, -5, -5,-10,-10,-20]
]

kingMiddleGame = [
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-20,-30,-30,-40,-40,-30,-30,-20],
    [-10,-20,-20,-20,-20,-20,-20,-10],
    [20, 20,  0,  0,  0,  0, 20, 20],
    [20, 30, 10,  0,  0, 10, 30, 20]
]

kingEndGame = [
    [-50,-40,-30,-20,-20,-30,-40,-50],
    [-30,-20,-10,  0,  0,-10,-20,-30],
    [-30,-10, 20, 30, 30, 20,-10,-30],
    [-30,-10, 30, 40, 40, 30,-10,-30],
    [-30,-10, 30, 40, 40, 30,-10,-30],
    [-30,-10, 20, 30, 30, 20,-10,-30],
    [-30,-30,  0,  0,  0,  0,-30,-30],
    [-50,-30,-30,-30,-30,-30,-30,-50]
]

nodesVisited = [0]

In [3]:
def ChessBoardSetup():
    # initialize and return a chess board - create a 2D 8x8 array that has the value for each cell
    # USE the following characters for the chess pieces - lower-case for BLACK and upper-case for WHITE
    # . for empty board cell
    # p/P for pawn
    # r/R for rook
    # t/T for knight
    # b/B for bishop
    # q/Q for queen
    # k/K for king
    chessBoard = [
        ['r', 't', 'b', 'q', 'k', 'b', 't', 'r'],
        ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
        ['R', 'T', 'B', 'Q', 'K', 'B', 'T', 'R']
    ]
    return chessBoard
    
#print the chessboard
def DrawBoard(chessBoard):
    for i in range(8):
        for j in range(8):
            print("%c   " % (chessBoard[i][j]), end =" ")
        print('\n')
    print('\n')

#check if a rook has moved
def rookMoved(chess, fromSquare):
    x, y = fromSquare
    if x == 0 and y == 0:
        chess.bLeftRookMoved = True
    if x == 0 and y == 7:
        chess.bRightRookMoved = True
    if x == 7 and y == 0:
        chess.wLeftRookMoved = True
    if x == 7 and y == 7:
        chess.wRightRookMoved = True

#perform a castle move
def castle(chess, fromSquare, toSquare):
    chessBoard = chess.board
    fromValue = chessBoard[fromSquare[0]][fromSquare[1]]
    toValue = chessBoard[toSquare[0]][toSquare[1]]
    if fromValue == 'K':
        chessBoard[7][4] = '.'
        if toSquare[1] > fromSquare[1]:
            chessBoard[7][6] = 'K'
            chessBoard[7][5] = 'R'
            chessBoard[7][7] = '.'
        else:
            chessBoard[7][2] = 'K'
            chessBoard[7][3] = 'R'
            chessBoard[7][0] = '.'
    else:
        chessBoard[0][4] = '.'
        if toSquare[1] > fromSquare[1]:
            chessBoard[0][6] = 'k'
            chessBoard[0][5] = 'r'
            chessBoard[0][7] = '.'
        else:
            chessBoard[0][2] = 'k'
            chessBoard[0][3] = 'r'
            chessBoard[0][0] = '.'

#undo a castle
def undoCastle(chess, fromSquare, toSquare):
    chessBoard = chess.board
    fromX, fromY = fromSquare
    toX, toY = toSquare
    fromValue = chessBoard[fromSquare[0]][fromSquare[1]]
    toValue = chessBoard[toSquare[0]][toSquare[1]]
    if fromX == 7 and fromY == 4:
        chess.whiteKingMoved = False
        chessBoard[7][4] = 'K'
        if toSquare[1] > fromSquare[1]:
            chess.wRightRookMoved = False
            chessBoard[7][7] = 'R'
            chessBoard[7][5] = '.'
            chessBoard[7][6] = '.'
        else:
            chess.wLeftRookMoved = False
            chessBoard[7][2] = '.'
            chessBoard[7][3] = '.'
            chessBoard[7][0] = 'R'
    else:
        chess.blackKingMoved = False
        chessBoard[0][4] = 'k'
        if toSquare[1] > fromSquare[1]:
            chess.bRightRookMoved = False
            chessBoard[0][6] = '.'
            chessBoard[0][5] = '.'
            chessBoard[0][7] = 'r'
        else:
            chess.bLeftRookMoved = False
            chessBoard[0][2] = '.'
            chessBoard[0][3] = '.'
            chessBoard[0][0] = 'r'

#move the piece on the from square to the to square
def MovePiece(chess, fromSquare, toSquare):
    chessBoard = chess.board
    if chessBoard[fromSquare[0]][fromSquare[1]] == 'k':
        chess.blackKingMoved = True
    if chessBoard[fromSquare[0]][fromSquare[1]] == 'K':
        chess.whiteKingMoved = True
    if chessBoard[fromSquare[0]][fromSquare[1]] == 'k' and chessBoard[toSquare[0]][toSquare[1]] == 'r':
        castle(chess, fromSquare, toSquare)
        rookMoved(chess, toSquare)
        return 'castle'
    if chessBoard[fromSquare[0]][fromSquare[1]] == 'K' and chessBoard[toSquare[0]][toSquare[1]] == 'R':
        castle(chess, fromSquare, toSquare)
        rookMoved(chess, toSquare)
        return 'castle'
    if chessBoard[fromSquare[0]][fromSquare[1]] == 'r' or chessBoard[fromSquare[0]][fromSquare[1]] == 'R':
        rookMoved(chess, fromSquare)
    if chessBoard[toSquare[0]][toSquare[1]] == 'r' or chessBoard[toSquare[0]][toSquare[1]] == 'R':
        rookMoved(chess, toSquare)
    tmp = chessBoard[toSquare[0]][toSquare[1]]
    chessBoard[toSquare[0]][toSquare[1]] = chessBoard[fromSquare[0]][fromSquare[1]]
    chessBoard[fromSquare[0]][fromSquare[1]] = '.'
    if chessBoard[toSquare[0]][toSquare[1]] == 'p' and toSquare[0] == 7:
        chessBoard[toSquare[0]][toSquare[1]] = 'q'
    if chessBoard[toSquare[0]][toSquare[1]] == 'P' and toSquare[0] == 0:
        chessBoard[toSquare[0]][toSquare[1]] = 'Q'
    return tmp

# Chess Rules

In [4]:
# return True if the input move (from-square and to-square) is legal, else False
# this is the KEY function which contains the rules for each piece type 
def IsMoveLegal(chess, fromSquare, toSquare):
    # input is from-square and to-square
    # use the input and the board to get the from-piece and to-piece
    chessBoard = chess.board
    if fromSquare[0] == toSquare[0] and fromSquare[1] == toSquare[1]:
        return False
    fromValue = chessBoard[fromSquare[0]][fromSquare[1]]
    toValue = chessBoard[toSquare[0]][toSquare[1]]
    if fromValue == '.':
        return False
    if IsPieceWhite(fromValue):
        color = 'white'
    else:
        color = 'black'
    if fromValue == 'p' or fromValue == 'P':
        if color == 'white':
            ## case - pawn wants to move one step forward (or backward if white)
            if toValue == '.' and fromSquare[1] == toSquare[1] and toSquare[0] == fromSquare[0] - 1: 
                return True
            ## case - pawn can move two spaces forward (or backward if white) ONLY if pawn on starting row
            elif toValue == '.' and fromSquare[0] == 6 and toSquare[0] == 4 and fromSquare[1] == toSquare[1]:
                if IsClearPath(chess, fromSquare, toSquare): # a clear path exists between from-square and to-square
                    return True
            ## case - pawn attacks the enemy piece if diagonal
            if toValue != '.' and IsPieceEnemy(fromValue, toValue) and toSquare[0] == fromSquare[0] - 1 and (toSquare[1] == fromSquare[1] + 1 or toSquare[1] == fromSquare[1] - 1):
                return True
        else:
            ## case - pawn wants to move one step forward (or backward if white)
#             print("From Square is on: %d, %d; its value is %s" % (fromSquare[0], fromSquare[1], chessBoard[fromSquare[0]][fromSquare[1]]))
#             print("To Square is on: %d, %d; its value is %s" % (toSquare[0], toSquare[1], chessBoard[toSquare[0]][toSquare[1]]))
            if toValue == '.' and fromSquare[1] == toSquare[1] and toSquare[0] == fromSquare[0] + 1: 
                return True
            ## case - pawn can move two spaces forward (or backward if white) ONLY if pawn on starting row
            elif toValue == '.' and fromSquare[0] == 1 and toSquare[0] == 3 and fromSquare[1] == toSquare[1]:
                if IsClearPath(chess, fromSquare, toSquare): # a clear path exists between from-square and to-square
                    return True
            ## case - pawn attacks the enemy piece if diagonal
            if toValue != '.' and IsPieceEnemy(fromValue, toValue) and toSquare[0] == fromSquare[0] + 1 and (toSquare[1] == fromSquare[1] + 1 or toSquare[1] == fromSquare[1] - 1):
                return True

    elif fromValue == 'r' or fromValue == 'R':
        if toSquare[0] == fromSquare[0] or toSquare[1] == fromSquare[1]:
            if toValue == '.' or IsPieceEnemy(fromValue, toValue):
                if IsClearPath(chess, fromSquare, toSquare):
                    return True

    elif fromValue == 'b' or fromValue == 'B':
        if toSquare[0] - fromSquare[0] != 0:
            if abs((toSquare[1] - fromSquare[1]) / (toSquare[0] - fromSquare[0])) == 1:
                if toValue == '.' or IsPieceEnemy(fromValue, toValue):
                    if IsClearPath(chess, fromSquare, toSquare):
                        return True

    elif fromValue == 'q' or fromValue == 'Q':
        if toSquare[0] == fromSquare[0] or toSquare[1] == fromSquare[1]:
            if toValue == '.' or IsPieceEnemy(fromValue, toValue):
                if IsClearPath(chess, fromSquare, toSquare):
                    return True
        # SAME as "bishop"
        if toSquare[0] - fromSquare[0] != 0:
            if abs((toSquare[1] - fromSquare[1]) / (toSquare[0] - fromSquare[0])) == 1:
                if toValue == '.' or IsPieceEnemy(fromValue, toValue):
                    if IsClearPath(chess, fromSquare, toSquare):
                        return True

    elif fromValue == 't' or fromValue == 'T':
        # calculate the col-diff = to-square-col - from-square-col
        colDiff = abs(toSquare[1] - fromSquare[1])
        # calculate the row-diff = to-square-row - from-square-row
        rowDiff = abs(toSquare[0] - fromSquare[0])
        if toValue == '.' or IsPieceEnemy(fromValue, toValue):
            if colDiff == 1 and rowDiff == 2:
                return True
            if colDiff == 2 and rowDiff == 1:
                return True

    elif fromValue == 'k' or fromValue == 'K':
        #can the king castle
        if fromValue == 'K' and toValue == 'R':
            if IsClearPath(chess, fromSquare, toSquare):
                if chess.whiteKingMoved == False:
                    if toSquare[1] > fromSquare[1] and not chess.wRightRookMoved:
                        return True
                    if toSquare[1] < fromSquare[1] and not chess.wLeftRookMoved:
                        return True
        if fromValue == 'k' and toValue == 'r':
            if IsClearPath(chess, fromSquare, toSquare):
                if chess.blackKingMoved == False:
                    if toSquare[1] > fromSquare[1] and not chess.bRightRookMoved:
                        return True
                    if toSquare[1] < fromSquare[1] and not chess.bLeftRookMoved:
                        return True
        #king wants to move 
        # calculate the col-diff = to-square-col - from-square-col
        colDiff = abs(toSquare[1] - fromSquare[1])
        # calculate the row-diff = to-square-row - from-square-row
        rowDiff = abs(toSquare[0] - fromSquare[0])
        if toValue == '.' or IsPieceEnemy(fromValue, toValue):
            if colDiff == 1 and rowDiff == 0:
                return True
            if colDiff == 0 and rowDiff == 1:
                return True
            if colDiff == 1 and rowDiff == 1:
                return True
    #move is not legal
    return False



# gets a list of legal moves for a given piece
# input = from-square
# output = list of to-square locations where the piece can move to
def GetListOfLegalMoves(chess, curPlayer, fromSquare):
    # input is the current player and the given piece as the from-square
    # initialize the list of legal moves, i.e., to-square locations to []
    # go through all squares on the board
    chessBoard = chess.board
    legalMoves = []
    for i in range(8):
        for j in range(8):
            toSquare = (i, j)
            if IsMoveLegal(chess, fromSquare, toSquare):
                # call DoesMovePutPlayerInCheck() with input as from-square and to-square and save the returned value
                # if returned value is False
                if not DoesMovePutPlayerInCheck(chess, curPlayer, fromSquare, toSquare):
                    legalMoves.append(toSquare)
    return legalMoves



# gets a list of all pieces for the current player that have legal moves
def GetPiecesWithLegalMoves(chess, curPlayer):
    # initialize the list of pieces with legal moves to []
    pieces = []
    legalMoves = []
    chessBoard = chess.board
    # go through all squares on the board
    for i in range(8):
        for j in range(8):
            curSquare = (i, j)
            if curPlayer == 'white':
                if not IsPieceWhite(chessBoard[i][j]):
                    continue
            elif curPlayer == 'black':
                if IsPieceWhite(chessBoard[i][j]):
                    continue
            curMoves = GetListOfLegalMoves(chess, curPlayer, curSquare)
            if len(curMoves) > 0:
                pieces.append(curSquare)
                legalMoves.append(curMoves)
    return pieces
#     return [pieces, legalMoves]


# returns True if the current player is in checkmate, else False
def IsCheckmate(chess, curPlayer):
    pieces = GetPiecesWithLegalMoves(chess, curPlayer)
    if len(pieces) == 0 and IsInCheck(chess, curPlayer):
        return True
    return False

def IsStalemate(chess, curPlayer):
    chessBoard = chess.board
    if curPlayer == 'white':
        enemyPlayer = 'black'
    else:
        enemyPlayer = 'white'
    return len(GetPiecesWithLegalMoves(chess, curPlayer)) == 0 and not IsCheckmate(chess, enemyPlayer)

def IsGameOver(chess, curPlayer):
    if IsCheckmate(chess, 'white') or IsCheckmate(chess, 'black'):
        return True
    if IsStalemate(chess, curPlayer):
        return True
    return False

# returns True if the given player is in Check state
def IsInCheck(chess, curPlayer):
    # find given player's King's location = king-square
    chessBoard = chess.board
    kingSquare = None
    for i in range(8):
        for j in range(8):
            if curPlayer == 'white' and chessBoard[i][j] == 'K':
                kingSquare = (i, j)
                break
            elif curPlayer == 'black' and chessBoard[i][j] == 'k':
                kingSquare = (i, j)
                break
    if kingSquare == None:
        return True
    kingValue = chessBoard[kingSquare[0]][kingSquare[1]]
    # go through all squares on the board
    for i in range(8):
        for j in range(8):
            # if there is a piece at that location and that piece is of the enemy team
            if chessBoard[i][j] != '.' and IsPieceEnemy(kingValue, chessBoard[i][j]):
                # call IsMoveLegal() for the enemy player from that square to the king-square
                # if the value returned is True
                if IsMoveLegal(chess, (i, j), kingSquare):
                    return True
    return False



# helper function to figure out if a move is legal for straight-line moves (rooks, bishops, queens, pawns)
# returns True if the path is clear for a move (from-square and to-square), non-inclusive
def IsClearPath(chess, fromSquare, toSquare):
    # given the move (from-square and to-square)
    newFromSquare = None
    chessBoard = chess.board
    if chebyshev(fromSquare, toSquare) == 1:
        return True
    if fromSquare[0] == toSquare[0] and fromSquare[1] == toSquare[1]:
        return True
    else:
        # if to-square is in the +ve vertical direction from from-square
        if toSquare[0] > fromSquare[0] and toSquare[1] == fromSquare[1]:
            # new-from-square = next square in the +ve vertical direction
            newFromSquare = (fromSquare[0] + 1, fromSquare[1])
        # else if to-square is in the -ve vertical direction from from-square
        elif toSquare[0] < fromSquare[0] and toSquare[1] == fromSquare[1]:
            # new-from-square = next square in the -ve vertical direction
            newFromSquare = (fromSquare[0] - 1, fromSquare[1])
        # else if to-square is in the +ve horizontal direction from from-square
        elif toSquare[1] > fromSquare[1] and toSquare[0] == fromSquare[0]:
            # new-from-square = next square in the +ve horizontal direction
            newFromSquare = (fromSquare[0], fromSquare[1] + 1)
        # else if to-square is in the -ve horizontal direction from from-square
        elif toSquare[1] < fromSquare[1] and toSquare[0] == fromSquare[0]:
            newFromSquare = (fromSquare[0], fromSquare[1] - 1)
        # else if to-square is in the SE diagonal direction from from-square
        elif toSquare[0] > fromSquare[0] and toSquare[1] > fromSquare[1]:
            if toSquare[0] - fromSquare[0] != 0:
                if abs((toSquare[1] - fromSquare[1]) / (toSquare[0] - fromSquare[0])) == 1:
                    # new-from-square = next square in the SE diagonal direction
                    newFromSquare = (fromSquare[0] + 1, fromSquare[1] + 1)
        elif toSquare[0] > fromSquare[0] and toSquare[1] < fromSquare[1]:
            if toSquare[0] - fromSquare[0] != 0:
                if abs((toSquare[1] - fromSquare[1]) / (toSquare[0] - fromSquare[0])) == 1:
                    # new-from-square = next square in the SW diagonal direction
                    newFromSquare = (fromSquare[0] + 1, fromSquare[1] - 1)
        # else if to-square is in the NE diagonal direction from from-square
        elif toSquare[0] < fromSquare[0] and toSquare[1] > fromSquare[1]:
            if toSquare[0] - fromSquare[0] != 0:
                if abs((toSquare[1] - fromSquare[1]) / (toSquare[0] - fromSquare[0])) == 1:
                    newFromSquare = (fromSquare[0] - 1, fromSquare[1] + 1)
        # else if to-square is in the NW diagonal direction from from-square
        elif toSquare[0] < fromSquare[0] and toSquare[1] < fromSquare[1]:
            if toSquare[0] - fromSquare[0] != 0:
                if abs((toSquare[1] - fromSquare[1]) / (toSquare[0] - fromSquare[0])) == 1:
                    newFromSquare = (fromSquare[0] - 1, fromSquare[1] - 1)
    # if new-from-square is not empty
    if newFromSquare == None or chessBoard[newFromSquare[0]][newFromSquare[1]] != '.':
        return False
    else:
        return IsClearPath(chess, newFromSquare, toSquare)


# makes a hypothetical move (from-square and to-square)
# returns True if it puts current player into check
def DoesMovePutPlayerInCheck(chess, curPlayer, fromSquare, toSquare):
    chessBoard = chess.board
    # given the move (from-square and to-square), find the 'from-piece' and 'to-piece'
    fromValue = chessBoard[fromSquare[0]][fromSquare[1]]
    toValue = chessBoard[toSquare[0]][toSquare[1]]
    # make the move temporarily by changing the 'board'
    chessBoard[fromSquare[0]][fromSquare[1]] = '.'
    chessBoard[toSquare[0]][toSquare[1]] = fromValue
    # Call the IsInCheck() function to see if the 'player' is in check - save the returned value
    inCheck = IsInCheck(chess, curPlayer)
    # Undo the temporary move
    chessBoard[fromSquare[0]][fromSquare[1]] = fromValue
    chessBoard[toSquare[0]][toSquare[1]] = toValue
    return inCheck
    
#determines if piece 2 is an enemy to piece 1
def IsPieceEnemy(piece1, piece2):
    if piece2 == '.':
        return False
    if IsPieceWhite(piece1):
        if IsPieceWhite(piece2):
            return False
        else:
            return True
    else:
        if IsPieceWhite(piece2):
            return True
        else: 
            return False

# returns true if the value belongs to the white pieces 
def IsPieceWhite(piece):
    return piece.isupper() and piece != '.'
#returns chebyshev distance between two squares
def chebyshev(fromSquare, toSquare):
    return max(abs(toSquare[0] - fromSquare[0]), abs(toSquare[1] - fromSquare[1]))

# "Artificial Intelligence"

This is an agent that simply selects a random move for the provided player and chessboard state

In [5]:
def GetRandomMove(chess, curPlayer):
    # pick a random piece and a random legal move for that piece
#     piecesWLegalMoves = GetPiecesWithLegalMoves(chess, curPlayer)
#     pieces = piecesWLegalMoves[0]
#     moves = piecesWLegalMoves[1]
#     if len(pieces) == 0:
#         return None
    pieces = GetPiecesWithLegalMoves(chess, curPlayer)
#     tmp = random.randrange(len(pieces))
#     piece = pieces[tmp]
#     pieceMoves = moves[tmp]
#     return pieceMoves[random.randrange(len(pieceMoves))]
    if len(pieces) == 0:
        return None
    piece = pieces[random.randrange(len(pieces))]
    moves = GetListOfLegalMoves(chess, curPlayer, piece)
    return [piece, moves[random.randrange(len(moves))]]

# Minmax Artficial Intelligence

This agent implents the minimax algortihm with alpha beta prunning to select the best move
for a given player and state. 

In [20]:
#evalutaion function used by agent to determine its score, checks using piece weights and positional values
#giving the agent situational and some decebt positional awareness. The openings are odd but sometimes you can 
#even see the agent perform some common openings seen in chess proving some positional awarness IMO
#Could defintely be better as this is a pretty basic eval function which is moticible in its performance
def evl(chessBoard, curPlayer):
    # this function will calculate the score on the board, if a move is performed
    # give score for each of piece and calculate the score for the chess board
    whiteScore = 0
    blackScore = 0
    whiteHasQueen = False
    blackHasQueen = False
    wMinorPieces = 0
    bMinorPieces = 0
    score = 0
    whiteKing = None
    blackKing = None
    for i in range(8):
        for j in range(8):
            value = chessBoard[i][j]
            if value == 'p' or value == 'P':
                if value == 'P':
                    whiteScore += 100
                    whiteScore += pawns[i][j]
                elif value == 'p':
                    blackScore += 100
                    blackScore += pawns[-i - 1][j]
            if value == 'b' or value == 'B' or value == 't' or value == 'T':
                if value == 'B':
                    whiteScore += 330
                    whiteScore += bishops[i][j]
                    wMinorPieces += 1
                elif value == 'b':
                    blackScore += 330
                    blackScore += bishops[-i - 1][j]
                    bMinorPieces += 1
                elif value == 'T':
                    whiteScore += 320
                    whiteScore += knights[i][j]
                    wMinorPieces += 1
                elif value == 't':
                    blackScore += 320
                    blackScore += knights[-i - 1][j]
                    bMinorPieces += 1
            if value == 'r' or value == 'R':
                if value == 'R':
                    whiteScore += 500
                    whiteScore += rooks[i][j]
                elif value == 'r':
                    blackScore += 500
                    blackScore += rooks[-i - 1][j]
            if value == 'q' or value == 'Q':
                if value == 'Q':
                    whiteScore += 900
                    whiteScore += queens[i][j]
                    whiteHasQueen = True
                elif value == 'q':
                    blackScore += 900
                    blackScore += queens[-i - 1][j]
                    blackHasQueen = True
            if value == 'k' or value == 'K':
                if value == 'K':
                    whiteKing = (i, j)
                if value == 'k':
                    blackKing = (i, j)
    if whiteKing != None:
        if (whiteHasQueen and blackHasQueen and wMinorPieces <= 1 and bMinorPieces <= 1) or (not whiteHasQueen and not blackHasQueen):
            whiteScore += 20000
            whiteScore += kingEndGame[i][j]
        else:
            whiteScore += 20000
            whiteScore += kingMiddleGame[i][j]
    if blackKing != None:
        if (whiteHasQueen and blackHasQueen and wMinorPieces <= 1 and bMinorPieces <= 1) or (not whiteHasQueen and not blackHasQueen):
            blackScore += 20000
            blackScore += kingEndGame[-i - 1][j]
        else:
            blackScore += 20000
            blackScore += kingMiddleGame[-i - 1][j]
    if curPlayer == 'white':
        return whiteScore - blackScore
    else:
        return blackScore - whiteScore

#pre process moves. ordering them by best chance to be best move (i.e highest score for player after performing)
def preProcessmoves(chess, moves):
    chessBoard = chess.board
    #get every move we are about to try and order them from the best resulting score to least
    orderedMoves = []
    #order the moves by best chance of being the best move (captures first then best res from making move)
    for i in range(len(moves[0])):
        #cur piece
        piece = moves[0][i]
        for j in range(len(moves[1][i])):
            #make move
            pieceVal = chessBoard[piece[0]][piece[1]]
            move = moves[1][i][j]
            lostPiece = MovePiece(chess, piece, move)
            #get eval of move
            res = evl(chessBoard, 'white')
            #undo move
            if lostPiece == 'castle':
                undoCastle(chess, piece, move)
            else:
                MovePiece(chess, move, piece)
                chessBoard[move[0]][move[1]] = lostPiece
                if pieceVal == 'P' and chessBoard[piece[0]][piece[1]] == 'Q':
                    chessBoard[piece[0]][piece[1]] = 'P'
            #if move is capture, add score to move to front
#             if chessBoard[move[0]][move[1]] != '.' and IsPieceEnemy(pieceVal, chessBoard[move[0]][move[1]]):
#                 res += 20000
            orderedMoves.append([piece, move, res])
    return orderedMoves
    
def maxi(chess, curPlayer, depth, alpha, beta):
    chessBoard = chess.board
    nodesVisited[0] += 1
    if depth == 0 or IsGameOver(chess, curPlayer):
        return []
    if curPlayer == 'white':
        enemyPlayer = 'black'
    else:
        enemyPlayer = 'white'
    
    pieces = GetPiecesWithLegalMoves(chess, curPlayer)
    orderedMoves = []
    captures = []
    noncaptures = []
    for piece in pieces:
        moves = GetListOfLegalMoves(chess, 'white', piece)
        for move in moves:
            if IsPieceEnemy(chess.board[piece[0]][piece[1]], chess.board[move[0]][move[1]]):
                captures.append([piece, move])
            else:
                noncaptures.append([piece, move])
    orderedMoves = captures + noncaptures
    bestMove = []
    bestRes = float('-inf')
    i = 0
#     for p in pieces:
#         orderedMoves = GetListOfLegalMoves(chess, 'white', p)
# #         print(orderedMoves)
    for item in orderedMoves:
#             print(item)
        #make move
        piece = item[0]
        move = item[1]
        pieceVal = chessBoard[piece[0]][piece[1]]
        #making curPlay move
        lostPiece = MovePiece(chess, piece, move)

        #if checkmate, best move...return 
        if IsCheckmate(chess, enemyPlayer):
            #undo move
            if lostPiece == 'castle':
                undoCastle(chess, piece, move)
            else:
                MovePiece(chess, move, piece)
                chessBoard[move[0]][move[1]] = lostPiece
                if pieceVal == 'P' and chessBoard[piece[0]][piece[1]] == 'Q':
                    chessBoard[piece[0]][piece[1]] = 'P'
            return [(piece, move)]

        #get best enemy move
        bestEnemyMove = mini(chess, enemyPlayer, depth - 1, alpha, beta)
        lostPieceFromEnemyMove = None

        #make enemy move, if there is one
        if len(bestEnemyMove) > 0:
            bestEnemyMove = bestEnemyMove[0]
            enemyPieceVal = chessBoard[bestEnemyMove[0][0]][bestEnemyMove[0][1]]
            lostPieceFromEnemyMove = MovePiece(chess, bestEnemyMove[0], bestEnemyMove[1])
        #get eval
        res = evl(chessBoard, curPlayer)
        if res > bestRes:
            bestRes = res
            bestMove = [(piece, move)]
#             item[2] = res

        ##undo enemy move if there is one
        if len(bestEnemyMove) > 0: 
            if lostPieceFromEnemyMove == 'castle':
                undoCastle(chess, bestEnemyMove[0], bestEnemyMove[1])
            else:
                MovePiece(chess, bestEnemyMove[1], bestEnemyMove[0])
                chessBoard[bestEnemyMove[1][0]][bestEnemyMove[1][1]] = lostPieceFromEnemyMove
                if enemyPieceVal == 'p' and chessBoard[bestEnemyMove[0][0]][bestEnemyMove[0][1]] == 'q':
                    chessBoard[piece[0]][piece[1]] = 'p'
        #undo cur move
        if lostPiece == 'castle':
            undoCastle(chess, piece, move)
        else:
            MovePiece(chess, move, piece)
            chessBoard[move[0]][move[1]] = lostPiece
            if pieceVal == 'P' and chessBoard[piece[0]][piece[1]] == 'Q':
                chessBoard[piece[0]][piece[1]] = 'P'
        #check for prune, update alpha
        alpha = max(alpha, res)
        if res > beta:
#             orderedMoves = sorted(orderedMoves, key=lambda x: x[2], reverse=True)
            return [(piece, move)]
    #return moves sorted order, best to worst, descending
#     orderedMoves = sorted(orderedMoves, key=lambda x: x[2], reverse=True)
    return bestMove




def mini(chess, curPlayer, depth, alpha, beta):
    chessBoard = chess.board
    nodesVisited[0] += 1
    if depth == 0 or IsGameOver(chess, curPlayer):
        return []
    if curPlayer == 'white':
        enemyPlayer = 'black'
    else:
        enemyPlayer = 'white'
    pieces = GetPiecesWithLegalMoves(chess, curPlayer)
    orderedMoves = []
    captures = []
    noncaptures = []
    for piece in pieces:
        moves = GetListOfLegalMoves(chess, 'black', piece)
        for move in moves:
            if IsPieceEnemy(chess.board[piece[0]][piece[1]], chess.board[move[0]][move[1]]):
                captures.append([piece, move])
            else:
                noncaptures.append([piece, move])
    orderedMoves = captures + noncaptures
    bestMove = []
    bestRes = float('inf')
    i = 0
    for item in orderedMoves:
        #make move 
        piece = item[0]
        move = item[1]
        pieceVal = chessBoard[piece[0]][piece[1]]
        lostPiece = MovePiece(chess, piece, move)

        #if checkmate, return
        if IsCheckmate(chess, enemyPlayer):
            #undo move
            if lostPiece == 'castle':
                undoCastle(chess, piece, move)
            else:
                MovePiece(chess, move, piece)
                chessBoard[move[0]][move[1]] = lostPiece
                if pieceVal == 'p' and chessBoard[piece[0]][piece[1]] == 'q':
                    chessBoard[piece[0]][piece[1]] = 'p'
            return [(piece, move)]

        #get best enemy move
        bestEnemyMove = maxi(chess, enemyPlayer, depth - 1, alpha, beta)
        lostPieceFromEnemyMove = None

        #make best enemy move, if there is one
        if len(bestEnemyMove) > 0:
            bestEnemyMove = bestEnemyMove[0]
            enemyPieceVal = chessBoard[bestEnemyMove[0][0]][bestEnemyMove[0][1]]
            lostPieceFromEnemyMove = MovePiece(chess, bestEnemyMove[0], bestEnemyMove[1])
        #evaluate move
        res = evl(chessBoard, 'white')
        #update res for move and best res if better than what has been found
        if res < bestRes:
            bestRes = res
            bestMove = [(piece, move)]
        #undo best enemy move
        if len(bestEnemyMove) > 0: 
            if lostPieceFromEnemyMove == 'castle':
                undoCastle(chess, bestEnemyMove[0], bestEnemyMove[1])
            else:
                MovePiece(chess, bestEnemyMove[1], bestEnemyMove[0])
                chessBoard[bestEnemyMove[1][0]][bestEnemyMove[1][1]] = lostPieceFromEnemyMove
                if enemyPieceVal == 'P' and chessBoard[bestEnemyMove[0][0]][bestEnemyMove[0][1]] == 'Q':
                    chessBoard[bestEnemyMove[0][0]][bestEnemyMove[0][1]] = 'P'
        #undo cur player move
        if lostPiece == 'castle':
            undoCastle(chess, piece, move)
        else:
            MovePiece(chess, move, piece)
            chessBoard[move[0]][move[1]] = lostPiece
            if pieceVal == 'p' and chessBoard[piece[0]][piece[1]] == 'q':
                chessBoard[piece[0]][piece[1]] = 'p'
        #check for prune, update beta
        beta = min(beta, res)
        if res < alpha:
#             orderedMoves = sorted(orderedMoves, key=lambda x: x[2])
            return [(piece, move)]
    return bestMove
    #return moves sorted from best to worst (ascending)
#     orderedMoves = sorted(orderedMoves, key=lambda x: x[2])
#     return orderedMoves


#Game Setup & Main Loop

(05 points)

Write code below to have a game-play between two AIs - Random vs MinMax. For each iteration, draw the board before and after a turn. 

In [21]:
#definition for n-ply minimax, will call 2-ply and 4-ply in cells below this one
#both will use alpha beta pruning and simple move ordering (testing 'best' moves first)
# in order to reduce the number of nodes explored 
def RandomVsNPly(n, clearOutput):
    sum = 0

    chess = Chess()
    chess.board = ChessBoardSetup()
    curPlayer = 'white'
    turns = 1
    N = 100
    movesPlayed = []
    # main game loop - while a player is not in checkmate or stalemate (<N turns)
    # below is the rough looping strategy
    while not IsCheckmate(chess, 'white') and not IsCheckmate(chess, 'black') and turns < N:
        # write code to take turns and move the piece
        chessBoard = chess.board
        x = time.perf_counter()
        whiteMoves = maxi(chess, 'white', n, float('-inf'), float('inf'))
#         print(whiteMoves)
        y = time.perf_counter()
        sum += (y - x)
        if len(whiteMoves) > 0:
            whiteMove = whiteMoves[0]
        #naive attempt to avoid repeating moves, not very good
        if len(movesPlayed) > 5:
            if whiteMove == movesPlayed[-4]:
                    if movesPlayed[-2] == movesPlayed[-6]:
                        if len(whiteMoves) > 1:
                            whiteMove = moves[1]
                        else:
                            whiteMove = GetRandomMove(chess, curPlayer)
        if whiteMove == None or IsCheckmate(chess, 'white'):
            break
        else:
            pieceMoved = chessBoard[whiteMove[0][0]][whiteMove[0][1]]
            pieceCaptured = chessBoard[whiteMove[1][0]][whiteMove[1][1]]
            if pieceMoved == 'K' and pieceCaptured == 'R':
                if whiteMove[0][1] > whiteMove[1][1]:
                    movesPlayed.append('long')
                if whiteMove[0][1] < whiteMove[1][1]:
                    movesPlayed.append('short')
            else:
                movesPlayed.append((whiteMove, pieceMoved, pieceCaptured))
            MovePiece(chess, whiteMove[0], whiteMove[1])
        blackMove = GetRandomMove(chess, 'black')
        if blackMove == None or IsCheckmate(chess, 'black'):
            break
        else:
#             print(blackMove)
            pieceMoved = chessBoard[blackMove[0][0]][blackMove[0][1]]
            pieceCaptured = chessBoard[blackMove[1][0]][blackMove[1][1]]
            if pieceMoved == 'k' and pieceCaptured == 'r':
                if whiteMove[0][1] > whiteMove[1][1]:
                    movesPlayed.append('long')
                if whiteMove[0][1] < whiteMove[1][1]:
                    movesPlayed.append('short')
            else:
                movesPlayed.append((blackMove, pieceMoved, pieceCaptured))
            MovePiece(chess, blackMove[0], blackMove[1])
        if clearOutput:
            clear_output()
        print("Turn: %d" % (turns))
        DrawBoard(chess.board)
        time.sleep(0.5)
        turns += 1
    clear_output()
    print("Turn: %d" % (turns))
    DrawBoard(chess.board)
    if IsCheckmate(chess, 'white'):
        print("Checkmate - Black wins!")
    elif IsCheckmate(chess, 'black'):
        print("Checkmate - White wins!")
    elif len(GetPiecesWithLegalMoves(chess, 'white')) == 0 and not IsCheckmate(chess, 'white'):
        print("Stalemate!")
    elif len(GetPiecesWithLegalMoves(chess, 'black')) == 0 and not IsCheckmate(chess, 'black'):
        print("Stalemate!")
    elif turns == N:
        print("Exceeded N turns")

    print("Average time to minimax move: %f\n" % (sum / turns))
    print("Game as PGN movelist (upload on lichess):")
    print(convertMovesToPGN(movesPlayed))
    return sum / 10
#     return movesPlayed

In [22]:
#2-ply (with alpha-beta pruning)
sum = 0
for i in range(5):
    sum += RandomVsNPly(2, clearOutput=True)
print(sum/10)

Turn: 19
r    t    .    B    .    Q    .    .    

.    .    .    .    .    .    p    .    

.    .    p    k    b    .    Q    t    

p    .    .    p    T    .    .    p    

.    p    .    P    .    .    .    .    

.    B    T    .    .    .    .    .    

P    P    P    .    .    P    P    P    

.    .    .    R    K    .    .    R    



Checkmate - White wins!
Average time to minimax move: 0.545723

Game as PGN movelist (upload on lichess):
1. Nbc3 hh6 2. Ngf3 bb6 3. dd4 Rhh7 4. ee4 ff5 5. exf5 dd5 6. Bfb5 Kef7 7. Nfe5 Kff6 8. Bcf4 hh5 9. Qdd3 cc6 10. Bba4 aa6 11. Bab3 aa5 12. Rad1 ee6 13. fxe6 Ngh6 14. Qxh7 bb5 15. Qhg6 Kfe7 16. Bfg5 Ked6 17. Bxd8 bb4 18. ee7 Bce6 19. exf8=Q 
0.5277549066595384


In [24]:
#4-ply (with alpha-beta pruning)
sum = 0
for i in range(1):
    sum += RandomVsNPly(4, clearOutput=True)
print(sum/10)

Turn: 27
.    .    .    Q    .    .    .    .    

k    .    .    .    Q    .    B    .    

.    .    p    .    .    .    .    .    

.    .    .    .    .    .    .    .    

.    .    .    .    .    .    .    .    

.    .    T    B    .    .    .    .    

P    P    P    .    .    P    P    P    

.    .    .    R    K    R    .    .    



Checkmate - White wins!
Average time to minimax move: 35.750201

Game as PGN movelist (upload on lichess):
1. Nbc3 Nba6 2. dd4 gg5 3. Bxg5 dd5 4. Ngf3 Bcg4 5. Nfe5 Rab8 6. Nxg4 Bfh6 7. Nxh6 ee5 8. Bxd8 Nxh6 9. Bdf6 exd4 10. Bxh8 Rbc8 11. Bxd4 cc6 12. Bxa7 bb6 13. Bxb6 dd4 14. Bxd4 Nab8 15. ee4 Nbd7 16. Bfd3 ff5 17. exf5 Nhg4 18. Qxg4 Ndf8 19. Qge4 Ked8 20. Rad1 Rcc7 21. Rhf1 Rcg7 22. Bxg7 Nfg6 23. fxg6 Kdc7 24. gxh7 Kcc8 25. hh8 Kcb7 26. Qhd8 Kba7 27. Qee7 
9.652554150409852


#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard



##Rubric

*   (10 points) Chess Board Setup
*   (50 points) Chess Rules Setup
*   (10 points) Random AI
*   (50 points) MinMax AI (2-ply)
*   (05 points) Game Main Loop - Random vs MinMax
*   (15 points) Extra Credit - 4-ply MinMax + alpha-beta pruning


